# Motifs Probabilisticos




Os **motifs** probabilísticos em Python referem-se a padrões ou sequências de eventos que ocorrem com uma certa probabilidade num conjunto de dados. 

Em bioinformática, por exemplo, os **motifs** podem representar padrões de nucleótidos em sequências de ADN, podendo também ser utilizados para encontrar regiões conservadas em sequências de ADN ou RNA.

As funções, **PWM** e **PSSM**, são ferramentas essenciais na análise de dados genómicos, na identificação de elementos regulatórios importantes nas sequências do ADN e na identificação de **motifs** conservados em regiões regulatórias do ADN. 

> ### Função PWM

A **função PWM** (Position Weight Matrix) é uma técnica amplamente utilizada na bioinformática para modelar padrões de consenso em sequências biológicas, como DNA ou proteínas. 

Consiste numa matriz que representa a frequência relativa de cada base em cada posição ao longo de um conjunto de sequências alinhadas.




Em primeiro lugar, procedeu-se à instalação da biblioteca **tabulate** que permite a utilização de um conjunto de ferramentas de forma a ter um output mais organizado.


In [ ]:
pip install tabulate

In [ ]:
'''
O *import* do módulo tabulate irá permitir a utilização das funções e classes fornecidas pelo *tabulate* para formatar e exibir dados tabulares
'''
from tabulate import tabulate

seqs = ['TGACTATACGTATGGTAGAT', 'ATCGTATACGTAGGTAGAC', 'TAGCTAGTCGTATGGTAGAT']
pseudo = 0

def pwm(seqs: list[str], pseudo: float = 0) -> list[dict[str, float]]:
  alfabeto = 'ACGT'

  """
  Calcula a matriz PWM (Matriz de Peso e Posição) para as sequências fornecidas

  Parâmetros
  -------------
  seqs : list[str]
      Recebe uma lista de strings que representam as sequências

  pseudo : float = 0
      Recebe um valor opcional, pseudo, que em caso de omissão é = 0


  Retorna
  -------------
  pwm_matrix : list[dict[str, float]]
      Retorna uma *lista de dicionários*, onde cada *dicionário* terá uma chave no formato de *string*, e um valor no formato de *float*

  """

  for seq in seqs:
    for idx, b in enumerate(seq):
      assert b in alfabeto, f'Caracter {b} na posição {idx} da sequência {seq} inválido!'

  pwm_matrix = [{b: (pos.count(b) + pseudo) / (len(seqs) + len(alfabeto) * pseudo)
    for b in alfabeto}
      for pos in zip(*seqs)]

  return pwm_matrix

resultado = pwm(seqs, pseudo)  # Associação do resultado da função a uma variável de forma a ser mais fácil imprimir o resultado

# Impressão da matriz formatada com o módulo tabulate
headers = ["Base"] + [f"Posição {i+1}" for i in range(len(seqs[0]))]
table = [[base] + [round(resultado[i][base], 3) for i in range(len(resultado))] for base in "ACGT"]
print(tabulate(table, headers))

In [30]:
import unittest

class TestBioInf(unittest.TestCase):

    def test_pwm(self):
        # Teste para sequências vazias
        self.assertEqual(pwm([]), [])
        
        # Teste para pseudocount zero
        self.assertEqual(pwm(['ATA'], pseudo=0), [{'A': 1.0, 'T': 0.0, 'C': 0.0, 'G': 0.0}])

        # Teste para pseudocount diferente de zero
        self.assertEqual(pwm(['ATA'], pseudo=1), [{'A': 0.75, 'T': 0.25, 'C': 0.0, 'G': 0.0}])

In [ ]:
def prob_seq(sequence, resultado):
    """
    Calcula a probabilidade de uma sequência utilizando como base a função PWM

    Parâmetros
    -------------
    sequence : str 
        A sequência de DNA 
     
    resultado : list[dict[str, float]] 
        Variável que contém o resultado da função PWM, onde cada dicionário representa as probabilidades para cada base em uma posição

    Retorna
    -------------
    probabilidade : float 
        A probabilidade da sequência com base na PWM
    
    """
    
    probabilidade = 1.0

    for position, base in enumerate(sequence):
        if base in resultado[position]:
            probabilidade *= resultado[position][base]
        else:
            probabilidade *= 0.01  # Atribui uma probabilidade mínima de 0.01 para bases ausentes.

    return probabilidade

seq = 'ATA'
probabilidade = prob_seq(seq, resultado)

print(f"A probabilidade da sequência '{seq}' é de: \n{probabilidade}")

In [40]:
import unittest

class TestBioInf(unittest.TestCase):

    def test_prob_seq(self):
        # Teste para pseudocount zero
        self.assertEqual(prob_seq('ATA', resultado), 0.03)

        # Teste para pseudocount diferente de zero
        self.assertEqual(prob_seq('ATA', resultado), 0.03)

In [ ]:
import re

def seq_provavel(seq, resultado):
  
  '''
  Calcula qual a Sequência mais provável

  Parâmetros
  -------------
  seq : str
    A sequência de DNA
  
  resultado : list[dict[str, float]] 
    Variável que contém o resultado da função PWM, onde cada dicionário representa as probabilidades para cada base em uma posição
  
  
  Retorna
  -------------
  str 
    A Sequência mais provável dentro da Sequência dada
  '''
  
  dicionario = {}
  for subset in re.findall('(?=(....))', seq):
    dicionario[subset] = (prob_seq(subset, resultado))
  return max(dicionario, key=dicionario.get)


seq = "CATTGT"
mais_provavel = seq_provavel(seq, resultado)

print(f'A Sequência mais provável, dentro da Sequência {seq}, é {mais_provavel}.')

In [39]:
import unittest

class TestBioInf(unittest.TestCase):

        def test_seq_provavel(self):
                # Teste para uma sequência específica
                self.assertEqual(seq_provavel("CATTGT", resultado), "CATT")

                # Teste para outra sequência
                self.assertEqual(seq_provavel("ATCGATCG", resultado), "ATCG")

                # Teste para uma sequência ausente das posições da PWM
                self.assertEqual(seq_provavel("GGGGGGGG", resultado), "GGGG")

>### Função PSSM

A **função PSSM** (Position-Specific Scoring Matrix) é uma extensão da PWM. 

Enquanto a PWM representa a frequência relativa das bases, a PSSM atribui uma pontuação para cada base em cada posição, levando em consideração a probabilidade logarítmica da ocorrência de uma base em relação a uma distribuição de probabilidade de fundo.

In [ ]:
import math

seqs = ['TGACTATACGTATGGTAGAT', 'ATCGTATACGTAGGTAGAC', 'TAGCTAGTCGTATGGTAGAT']

def pssm(seqs, pseudo = 1):

  """
  Calcula a Matriz de Pontuação de Posição Específica (PSSM) para um conjunto de sequências de DNA.

  Parâmetros
  -------------
  seqs : list[str]
    Lista de sequências de DNA.

  pseudo : float 
    Valor de pseudocount a ser adicionado para evitar problemas com probabilidades zero.


  Retorna
  -------------
  lista : list[dict[str, float]]
    Uma lista de dicionários que representa a PSSM.
  """

  bases = 'ATCG'
  lista = []

  for pos in list(zip(*seqs)):
    dicionario = {}
    for b in bases:
      # Fórmula da PSSM: log2((contagem da base + pseudocount) / (total de sequências + total de bases * pseudocount)) / 0.25
      dicionario[b] = math.log2((pos.count(b) + pseudo) / (len(seqs) + len(bases)*pseudo)) / 0.25

    lista.append(dicionario)
  return lista

matriz_pssm = pssm(seqs, pseudo = 1)

# O Table e o Headers já foram definidos para a função PWM

headers = ["Base"] + [f"Posição {i+1}" for i in range(len(seqs[0]))]
table = [[base] + [round(matriz_pssm[i][base], 3) for i in range(len(matriz_pssm))] for base in "ACGT"]
print(tabulate(table, headers))

In [43]:
import unittest

class TestBioInf(unittest.TestCase):

    def test_pssm(self):
        # Teste para sequências vazias
        self.assertEqual(pssm([]), [])

        # Teste para pseudocount zero
        self.assertEqual(pssm(['ATA'], pseudo=0), [{'A': 0.0, 'T': 0.0, 'C': 0.0, 'G': 0.0}])

        # Teste para pseudocount diferente de zero
        self.assertEqual(pssm(['ATA'], pseudo=1), [{'A': -0.415, 'T': 0.415, 'C': 0.415, 'G': -0.415}])